# Active Inference book

## Thomas Parr Giovanni Pezzulo_ Karl J_ Friston -- 2022 -- MIT Press

##  Mostly coding stuff will appear here

In [1]:
# Each chapter?  maybe there are only a few examples

Of course. Active inference can seem abstract at first, but seeing it in action with code can make the concepts much more concrete. Here are three basic Python program examples that demonstrate how to use a Bayesian framework for active inference.

These examples will use the `pymdp` library, a popular and well-documented Python package for active inference. It provides a clear structure for building the components of a generative model, which is at the heart of the Bayesian approach in active inference.

First, you'll need to install `numpy` and `pymdp`:

```bash
pip install numpy pymdp
```

-----

### Example 1: The "T-Maze" - A Classic Decision Task

In this classic experiment, an agent is at the bottom of a 'T' and must decide to go left or right. A cue at the beginning of the trial indicates where a reward is located. The agent wants to find the reward and has a preference for being at the reward location.

**The Bayesian Framework in Action:**

  * **Generative Model:** The agent has an internal model of how the world works. This model includes:
      * **Priors over states (D):** The agent's initial belief about where it is.
      * **Likelihood model (A):** The probability of receiving a particular observation given the true state of the world.
      * **Transition model (B):** The probability of transitioning to a new state given the current state and a chosen action.
      * **Priors over policies (E):** The agent's initial preferences for certain sequences of actions.
      * **Prior preferences (C):** The agent's goals, expressed as a probability distribution over desired outcomes.
  * **Inference:** The agent uses Bayesian inference (specifically, belief propagation or variational inference under the hood in `pymdp`) to update its beliefs about its current state based on observations.
  * **Action Selection:** The agent chooses the action that it predicts will minimize its "expected free energy" - a quantity that balances exploring to reduce uncertainty (epistemic value) and exploiting to achieve preferred outcomes (pragmatic value).

**Python Code:**

In [1]:
import numpy as np
from pymdp import agent

# 1. Define the Generative Model

# States: [Location (Up, Left, Right), Cue (Left, Right)]
# Observations: [Location Observation (See Up, See Left, See Right), Cue Observation (See Left, See Right)]
# Controls: [Move (Up, Left, Right)]

# Likelihood mapping (A matrix)
A = np.zeros((3, 3, 2))
A[0, 0, :] = 1.0  # If at 'Up', you observe 'Up'
A[1, 1, :] = 1.0  # If at 'Left', you observe 'Left'
A[2, 2, :] = 1.0  # If at 'Right', you observe 'Right'

A_cue = np.zeros((2, 3, 2))
A_cue[0, :, 0] = 1.0 # If cue is 'Left', you observe 'Left'
A_cue[1, :, 1] = 1.0 # If cue is 'Right', you observe 'Right'

A_list = [A, A_cue]

# Transition model (B matrix)
B = np.zeros((3, 3, 3))
B[0, 0, 0] = 1.0 # From 'Up', taking 'Up' action, stay 'Up'
B[1, 0, 1] = 1.0 # From 'Up', taking 'Left' action, go to 'Left'
B[2, 0, 2] = 1.0 # From 'Up', taking 'Right' action, go to 'Right'
B[:, 1, :] = np.expand_dims(np.eye(3)[:, 1], axis=1) # Stay at Left
B[:, 2, :] = np.expand_dims(np.eye(3)[:, 2], axis=1) # Stay at Right

B_cue = np.tile(np.eye(2), (3, 1, 1)).T
B_list = [B, B_cue]

# Prior preferences (C vector) - The agent wants to be at the rewarded location
# Let's say the agent wants the 'Left' reward
C = np.zeros(3)
C[1] = 1.0 # High preference for being at the 'Left' location

# 2. Create the Agent
my_agent = agent.Agent(A=A_list, B=B_list, C=C)

# 3. Run the simulation for one trial
# Initial observation: The agent sees the cue pointing 'Left'
observation = [0, 0] # Sees 'Up' location, sees 'Left' cue

# The agent infers its current state based on the observation
qs = my_agent.infer_states(observation)
print(f"Belief about current location after seeing cue: {qs[0]}")
print(f"Belief about the cue's direction: {qs[1]}")

# The agent infers the best policy (sequence of actions)
q_pi, _ = my_agent.infer_policies()
print(f"Beliefs over policies: {q_pi}")

# The agent selects an action
action = my_agent.sample_action()
print(f"Action taken: {action}")

TypeError: A matrix must be a numpy array

-----

### Example 2: Grid World Navigation

Here, an agent exists in a simple 2x2 grid world and wants to navigate to a specific goal location.

**The Bayesian Framework in Action:**

This example further highlights how the agent's beliefs about its own position are updated probabilistically. If the agent isn't certain where it is, it might take an action to confirm its location before heading to the goal.

**Python Code:**


In [ ]:
import numpy as np
from pymdp import agent, utils

# 1. Define the Generative Model
# States: [Location (0, 1, 2, 3)]
# Observations: [Location Observation (0, 1, 2, 3)]
# Controls: [Move (Stay, Up, Down, Left, Right)]

grid_size = 2
num_states = grid_size * grid_size
num_obs = num_states
num_controls = 5

# Likelihood mapping (A matrix) - Perfect observation of location
A = np.eye(num_states)

# Transition model (B matrix)
B = np.zeros((num_states, num_states, num_controls))

for s in range(num_states):
    B[s, s, 0] = 1.0 # Stay
    # Implement Up, Down, Left, Right transitions (with boundaries)
    row, col = np.unravel_index(s, (grid_size, grid_size))
    # Up
    next_row = max(0, row - 1)
    B[np.ravel_multi_index((next_row, col), (grid_size, grid_size)), s, 1] = 1.0
    # Down
    next_row = min(grid_size - 1, row + 1)
    B[np.ravel_multi_index((next_row, col), (grid_size, grid_size)), s, 2] = 1.0
    # Left
    next_col = max(0, col - 1)
    B[np.ravel_multi_index((row, next_col), (grid_size, grid_size)), s, 3] = 1.0
    # Right
    next_col = min(grid_size - 1, col + 1)
    B[np.ravel_multi_index((row, next_col), (grid_size, grid_size)), s, 4] = 1.0


# Prior preferences (C vector) - Goal is location 3 (bottom-right)
C = np.zeros(num_obs)
C[3] = 1.0

# 2. Create the Agent
my_agent = agent.Agent(A=A, B=B, C=C)

# 3. Run a step of the simulation
# Agent's initial belief is that it's at location 0 (top-left)
my_agent.qs = utils.onehot(0, num_states)
print(f"Initial belief about location: {my_agent.qs}")

# The agent gets an observation confirming it's at location 0
observation = 0

# Agent updates its beliefs (though they are already certain here)
qs = my_agent.infer_states(observation)

# Agent decides on the best policy
q_pi, _ = my_agent.infer_policies()
print(f"Beliefs over policies: {q_pi}") # Should favor moving right or down

# Agent takes an action
action = my_agent.sample_action()
print(f"Action taken (0=Stay, 1=Up, 2=Down, 3=Left, 4=Right): {action}")


-----

### Example 3: Perceptual Categorization - Is it a "Cat" or a "Dog"?

This example focuses on the perceptual aspect of active inference. The agent receives ambiguous sensory data and must infer the hidden cause of that data.

**The Bayesian Framework in Action:**

This scenario strips away the action component to purely showcase Bayesian belief updating. The agent has prior beliefs about whether it's more likely to see a cat or a dog. It then receives some sensory evidence (e.g., a feature like "pointy ears") and updates its beliefs to a posterior distribution.

**Python Code:**


In [ ]:
import numpy as np
from pymdp import utils

# 1. Define the Generative Model

# Hidden States: [Identity (Cat, Dog)]
# Observations: [Features (Pointy Ears, Floppy Ears)]

num_states = 2  # Cat, Dog
num_obs = 2     # Pointy, Floppy

# Likelihood mapping (A matrix)
# P(Observation | State)
A = np.zeros((num_obs, num_states))

# If it's a Cat, it's likely to have pointy ears
A[0, 0] = 0.8  # P(Pointy Ears | Cat)
A[1, 0] = 0.2  # P(Floppy Ears | Cat)

# If it's a Dog, it's more likely to have floppy ears
A[0, 1] = 0.3  # P(Pointy Ears | Dog)
A[1, 1] = 0.7  # P(Floppy Ears | Dog)

# 2. Define Prior Beliefs

# Prior beliefs about the hidden state (D vector)
# Let's say the agent thinks it's slightly more likely to see a dog
D = np.array([0.4, 0.6]) # P(Cat), P(Dog)
print(f"Prior belief (P(Cat), P(Dog)): {D}")

# 3. Perform Inference after an Observation

# The agent observes 'Pointy Ears' (observation index 0)
observation_index = 0

# Calculate the posterior belief using Bayes' rule (which is what infer_states does)
# Posterior = normalize(Likelihood * Prior)
likelihood_of_obs = A[observation_index, :]
posterior = utils.norm_dist(likelihood_of_obs * D)

print(f"Observation: Pointy Ears")
print(f"Likelihood of this observation given states: {likelihood_of_obs}")
print(f"Posterior belief (P(Cat), P(Dog)): {posterior}")

# Now, let's see what happens if it observes 'Floppy Ears'
observation_index = 1
likelihood_of_obs = A[observation_index, :]
posterior = utils.norm_dist(likelihood_of_obs * D)

print(f"\nObservation: Floppy Ears")
print(f"Likelihood of this observation given states: {likelihood_of_obs}")
print(f"Posterior belief (P(Cat), P(Dog)): {posterior}")


These examples provide a starting point for understanding how to implement active inference in Python. By manipulating the `A`, `B`, and `C` matrices, you can build increasingly complex and interesting agent behaviors, all grounded in the core principles of Bayesian inference.